In [1]:
queries = dict()
with open("../msmarco/queries.dev.tsv") as f:
    for line in f:
        qid, qtext = line.strip().split("\t")
        queries[qid] = qtext
        
collection = dict()
with open("../msmarco/collection.tsv") as f:
    for line in f:
        did, dtext = line.strip().split("\t")
        collection[did] = dtext

In [2]:
qrels = dict()
with open("../msmarco/qrels.dev.tsv") as f:
    for line in f:
        qid, _, did, rel = line.strip().split("\t")
        if qid not in qrels and int(rel) > 0:
            qrels[qid] = [did]
        elif int(rel) > 0:
            qrels[qid].append(did)

In [25]:

qset = set()
sample = dict()
run = dict()
with open("../msmarco/yifan_bm25_top1000.psg.dev.trec.trec") as f:
    for line in f:

        qid, _, did, rank, _, _ = line.strip().split("\t")
        if int(rank) > 100:
            continue
        if qid in qset:
            if did not in qrels[qid]:
                sample[qid]['negative'].append(collection[did])
        elif len(qset) < 100:
            sample[qid] = {'query': queries[qid], "positive": [collection[did] for did in qrels[qid]], 'negative': []}
            qset.add(qid)
            if did not in qrels[qid]:
                sample[qid]['negative'].append(collection[did])
                
        if  qid in qset and did in qrels[qid] and qid not in run and int(rank) <= 10:
            run[qid] = 1.0/(int(rank))

In [26]:
sample['1048585']

{'query': "what is paula deen's brother",
 'positive': ["Paula Deen and her brother Earl W. Bubba Hiers are being sued by a former general manager at Uncle Bubba'sâ\x80¦ Paula Deen and her brother Earl W. Bubba Hiers are being sued by a former general manager at Uncle Bubba'sâ\x80¦"],
 'negative': ["Racial scandals aren't always bad for business ... just ask Paula Deen's brother who says the embattled chef's restaurants have been slammed withâ\x80¦ Racial scandals aren't always bad for business ... just ask Paula Deen's brother who says the embattled chef's restaurants have been slammed withâ\x80¦",
  "Viewing Tweets won't unblock @Paula_Deen. 1  Paula Deenâ\x80\x8fVerified account @Paula_Deen 8h8 hours ago. 2  Paula Deenâ\x80\x8fVerified account @Paula_Deen 23h23 hours ago. 3  Paula Deenâ\x80\x8fVerified account @Paula_Deen Apr 4  9. Paula Deenâ\x80\x8fVerified account @Paula_Deen Apr 5  8. Paula Deenâ\x80\x8fVerified account @Paula_Deen Apr  8. Paula Deenâ\x80\x8fVerified account @Pa

In [27]:
from sentence_transformers import  LoggingHandler, SentenceTransformer, evaluation, util, models
model = SentenceTransformer( "../sentence-transformers/examples/training/ms_marco/output/train_bi-encoder-mnrl-distilbert-base-uncased-margin_3.0-2022-09-15_06-13-41")

In [28]:
import torch
all_doc_features = model.encode(sample['1048585']['negative'],
                            convert_to_tensor=True,
                            batch_size=16, output_value = None)

all_docs_embs = torch.stack([feature['sentence_embedding'] for feature in all_doc_features], dim = 0)

In [29]:
pos_doc_features = model.encode(sample['1048585']['positive'],
                            convert_to_tensor=True,
                            batch_size=16, output_value = None)

pos_docs_embs = torch.stack([feature['sentence_embedding'] for feature in pos_doc_features], dim = 0)

In [30]:
all_query_features = model.encode(sample['1048585']['query'],
                          convert_to_tensor=True,
                          batch_size=16,
                          show_progress_bar=True,
                          output_value = None) #self.show_progress_bar)


all_query_embs = all_query_features['sentence_embedding']

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
import json
open("../msmarco/samples_dev_bm25_tops_for_sbert.json", "w").write(json.dumps(list(sample.values())))

3821715

In [32]:
from sentence_transformers.util import cos_sim
cos_sim(all_query_embs, pos_docs_embs)

tensor([[0.6232]], device='cuda:0')

In [33]:
cos_sim(all_query_embs, all_docs_embs)

tensor([[ 0.3023,  0.1917,  0.4062,  0.2213,  0.6633,  0.2693,  0.4281,  0.2800,
          0.3382,  0.1706,  0.3609,  0.0178,  0.5630,  0.2704,  0.2209,  0.3152,
          0.3173,  0.2410,  0.3186,  0.2017,  0.2480,  0.1569,  0.0739,  0.2764,
          0.3848,  0.3449,  0.6747,  0.4362,  0.2676,  0.0798,  0.2178,  0.3054,
          0.2798,  0.2626,  0.0796,  0.2565,  0.3061,  0.1169,  0.2065,  0.0631,
          0.3362,  0.2869,  0.2757,  0.3047,  0.1186,  0.5206,  0.1264,  0.3824,
          0.1264,  0.3121,  0.3139,  0.2086,  0.3235,  0.3649,  0.3742,  0.2875,
          0.0316, -0.0228,  0.2329,  0.0292,  0.2891,  0.2479,  0.1272,  0.1655,
          0.1908,  0.0896,  0.2109,  0.0620,  0.1971,  0.2512,  0.2992,  0.2597,
          0.4145,  0.3174,  0.2230,  0.0899,  0.2608,  0.3987,  0.3915, -0.0995,
          0.2368,  0.1607,  0.1904,  0.0257,  0.2099,  0.1143,  0.2183,  0.2666,
          0.3276,  0.2031, -0.0101,  0.1426,  0.2214, -0.0365,  0.3266,  0.0147,
          0.1764,  0.0427,  

# TREC45

In [1]:
import ir_datasets

In [2]:
collection = dict()

with open("../trec45.doc.txt") as f:
    for line in f:
        collection[line.strip().split("\t")[1]] = line.strip().split("\t")[-1] 
        #did, dtext = line.strip().split("\t")
        #collection[did] = dtext

In [18]:
from collections import defaultdict
run = defaultdict(dict)

fname = [5]
for fn in fname:
    with open(f"../trec45/bm25.f{fn}.test.run.top150") as f:
        for line in f:
            qid, _, did, _, _, _ = line.strip().split(" ")
            if "docs" not in run[qid]:
                run[qid]["docs"] = [did]
            else:
                run[qid]["docs"].append(did) 

In [19]:
for fn in fname:
    dataset = ir_datasets.load(f"disks45/nocr/trec-robust-2004/fold{fn}")
    for q in dataset.queries_iter():
        run[q[0]]['qtext'] = q[1]

In [20]:
for fn in fname:
    dataset = ir_datasets.load(f"disks45/nocr/trec-robust-2004/fold{fn}")
    for q in dataset.qrels_iter():
        if 'posid' not in run[q[0]]:
            run[q[0]]['posid'] = []
        if q[2] > 0:
            run[q[0]]['posid'].append(q[1])

In [21]:
out = []
for qid in run:
    sample = dict()
    sample['query'] = run[qid]['qtext']
    sample['positive'] = []
    sample['negative'] = []
    if 'posid' not in run[qid] or 'docs' not in run[qid]:
        print("error")
        continue
    for did in run[qid]['posid']:
        if did in collection:
            sample['positive'].append(collection[did])
            
    for did in run[qid]['docs']:
        if did not in run[qid]['posid']:
            sample['negative'].append(collection[did])
    out.append(sample)

In [22]:
import json
open("../msmarco/samples_robust04_f5_bm25_tops_for_sbert.json", "w").write(json.dumps(out))

61441383

# generate top 150 data for reranking evaluation

In [6]:
import ir_datasets
fname = [1,2,3,4,5]
queries = dict()
for fn in fname:
    dataset = ir_datasets.load(f"disks45/nocr/trec-robust-2004/fold{fn}")
    for q in dataset.queries_iter():
        queries[q[0]] = q[1]

In [7]:
from collections import defaultdict
run = defaultdict(dict)
fo = open(f"../trec45/bm25.test.run.top150.tsv", "w")

for fn in fname:
    with open(f"../trec45/bm25.f{fn}.test.run.top150") as f:
        for line in f:
            qid, _, did, _, _, _ = line.strip().split(" ")
            fo.write(f"{qid}\t{did}\t{queries[qid]}\t{collection[did]}\n")
            fo.flush()
            
fo.close()

In [11]:
fo = open(f"../trec45/qrels.txt", "w")

fname = [1,2,3,4,5]
for fn in fname:
    dataset = ir_datasets.load(f"disks45/nocr/trec-robust-2004/fold{fn}")
    for q in dataset.qrels_iter():
        fo.write(f"{q[0]} Q0 {q[1]} {q[2]}\n")
        fo.flush()
        
fo.close()

# clueweb

In [ ]:
import ir_datasets
fname = [1,2,3,4,5]
queries = dict()
with open("/home/ec2-user/efs/clueweb/queries_proc.tsv") as f:
    for line in f:
        _, qid, qtext = line.strip().split("\t")
        queries[qid] = qtext

collection = dict()
with open("/home/ec2-user/efs/clueweb/documents_proc_top150.tsv") as f:
    for line in f:
        _, did, dtext = line.strip().split("\t")
        collection[did] = dtext

from collections import defaultdict
run = defaultdict(dict)
fo = open(f"/home/ec2-user/efs/clueweb/bm25.test.run.top150.tsv", "w")

for fn in fname:
    with open(f"/home/ec2-user/efs/clueweb/bm25.f{fn}.test.run.top150") as f:
        for line in f:
            qid, _, did, _, _, _ = line.strip().split(" ")
            fo.write(f"{qid}\t{did}\t{queries[qid]}\t{collection[did]}\n")
            fo.flush()
            
fo.close()


In [ ]:
qrels = dict()
with open("/home/ec2-user/efs/clueweb/qrels.clueweb09b.txt") as f:
    for line in f:
        qid, _, did, rel = line.strip().split(" ")

In [8]:
from collections import defaultdict
run = defaultdict(dict)

fname = [1]
for fn in fname:
    with open(f"/home/ec2-user/efs/clueweb/bm25.f{fn}.test.run.top150") as f:
        for line in f:
            qid, _, did, _, _, _ = line.strip().split(" ")
            if "docs" not in run[qid]:
                run[qid]["docs"] = [did]
            else:
                run[qid]["docs"].append(did) 
                
for qid in run:
    if 'posid' not in run[qid]:
        run[qid]['posid'] = []
    qrels[qid]

In [ ]:
for fn in fname:
    dataset = ir_datasets.load(f"disks45/nocr/trec-robust-2004/fold{fn}")
    for q in dataset.qrels_iter():
        if 'posid' not in run[q[0]]:
            run[q[0]]['posid'] = []
        if q[2] > 0:
            run[q[0]]['posid'].append(q[1])

In [ ]:
out = []
for qid in run:
    sample = dict()
    sample['query'] = run[qid]['qtext']
    sample['positive'] = []
    sample['negative'] = []
    if 'posid' not in run[qid] or 'docs' not in run[qid]:
        print("error")
        continue
    for did in run[qid]['posid']:
        if did in collection:
            sample['positive'].append(collection[did])
            
    for did in run[qid]['docs']:
        if did not in run[qid]['posid']:
            sample['negative'].append(collection[did])
    out.append(sample)